In [49]:
import numpy as np
import sys
import os

# add the libfranka library, insert the franka_interface path, then import
sys.path.insert(0, "/home/luke/libs/franka/franka_interface/build")
import pyfranka_interface

# create franka controller instance
franka = pyfranka_interface.Robot_("172.16.0.2", False, False)

# create identity matrix (must be floats!)
T = np.array(
  [[1,0,0,0],
  [0,1,0,0],
  [0,0,1,0],
  [0,0,0,1]],
  dtype=np.float32
)

# create up and down matrices
move = 2e-3
up = np.copy(T)
down = np.copy(T)
up[2,3] = move
down[2, 3] = -move

# define duration in seconds
duration = 0.75

In [17]:
Z_POS = 100e-3

In [14]:
move = 10e-3
up[2,3] = move
franka.move("relative", up, duration * 2)
Z_POS += move

In [26]:
move = 2e-3
down[2, 3] = -move
franka.move("relative", down, duration)
Z_POS -= move

In [47]:
state = franka.getState()

vec_str = "[{0:.8f}, {1:.8f}, {2:.8f}, {3:.8f}, {4:.8f}, {5:.8f}, {6:.8f}]".format(*state.q)

print("cal_X_mm =", vec_str)
print(f"cal_{Z_POS * 1000:.0f}_mm = {vec_str}")

cal_X_mm = [-0.27650041, -0.22718914, 0.17733962, -1.88203920, 0.00867977, 1.67482131, -0.54766548]
cal_132_mm = [-0.27650041, -0.22718914, 0.17733962, -1.88203920, 0.00867977, 1.67482131, -0.54766548]


In [50]:
cal = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 40, 50, 60, 70, 80, 90, 100]
# cal = [start + 2*x for x in range(25)]

start = cal[0]
end = cal[-1]

from time import sleep

Z_POS = start * 1e-3

for i in range(len(cal)):

  state = franka.getState()
  vec_str = "[{0:.8f}, {1:.8f}, {2:.8f}, {3:.8f}, {4:.8f}, {5:.8f}, {6:.8f}]".format(*state.q)
  print(f"cal_{Z_POS * 1000:.0f}_mm = {vec_str}")

  if i == len(cal) - 1: break
  if cal[i + 1] > end: break

  move = (cal[i + 1] - cal[i]) * 1e-3
  up[2,3] = move
  franka.move("relative", up, duration * 2)
  Z_POS += move

  sleep(0.3)

cal_0_mm = [-0.26363617, -0.05473505, 0.30020690, -1.90742241, 0.00734889, 1.87581724, 0.82313380]
cal_2_mm = [-0.26366258, -0.05472722, 0.30021328, -1.90369823, 0.00735250, 1.87093787, 0.82305917]
cal_4_mm = [-0.26371654, -0.05471687, 0.30021328, -1.89936716, 0.00741290, 1.86591738, 0.82294415]
cal_6_mm = [-0.26391464, -0.05504486, 0.30021328, -1.89472156, 0.00748206, 1.86090533, 0.82281917]
cal_8_mm = [-0.26396259, -0.05532080, 0.30021161, -1.88997358, 0.00757329, 1.85586071, 0.82272108]
cal_10_mm = [-0.26403608, -0.05563658, 0.30021328, -1.88528073, 0.00770713, 1.85078742, 0.82262177]
cal_12_mm = [-0.26422203, -0.05593439, 0.30021523, -1.88056663, 0.00779554, 1.84573231, 0.82252450]
cal_14_mm = [-0.26427300, -0.05619965, 0.30021331, -1.87574604, 0.00784266, 1.84068288, 0.82243819]
cal_16_mm = [-0.26438289, -0.05632581, 0.30021690, -1.87089863, 0.00788102, 1.83561542, 0.82236489]
cal_18_mm = [-0.26453586, -0.05657995, 0.30021523, -1.86600990, 0.00791936, 1.83057723, 0.82228960]
cal_2

In [34]:
# new calibrations 75DEGREE FINGERS BARE TABLE (NO WRIST) 4mm=light touch, 2mm=press
cal_0_mm = [-0.27387776, -0.02585006, 0.30729622, -1.89643083, 0.00676038, 1.87621824, 0.84037356]
cal_2_mm = [-0.27392039, -0.02572424, 0.30729622, -1.89337982, 0.00672408, 1.87142800, 0.84036733]
cal_4_mm = [-0.27400394, -0.02574353, 0.30729093, -1.88897988, 0.00673235, 1.86650059, 0.84031136]
cal_6_mm = [-0.27421166, -0.02623554, 0.30729463, -1.88446972, 0.00674949, 1.86159163, 0.84021751]
cal_8_mm = [-0.27424804, -0.02660902, 0.30729630, -1.88004050, 0.00677693, 1.85652403, 0.84012095]
cal_10_mm = [-0.27428154, -0.02698982, 0.30729559, -1.87545015, 0.00681192, 1.85143192, 0.84002916]
cal_12_mm = [-0.27454526, -0.02738393, 0.30730011, -1.87072350, 0.00689858, 1.84639242, 0.83994788]
cal_14_mm = [-0.27458083, -0.02776590, 0.30729819, -1.86601039, 0.00700295, 1.84137630, 0.83987414]
cal_16_mm = [-0.27461111, -0.02810298, 0.30729511, -1.86128947, 0.00710709, 1.83632393, 0.83981037]
cal_18_mm = [-0.27463627, -0.02841951, 0.30729817, -1.85654005, 0.00718308, 1.83133528, 0.83975061]
cal_20_mm = [-0.27467913, -0.02870156, 0.30729826, -1.85180733, 0.00724723, 1.82633607, 0.83968904]
cal_22_mm = [-0.27492752, -0.02881564, 0.30730066, -1.84703765, 0.00730904, 1.82137303, 0.83964258]
cal_24_mm = [-0.27495433, -0.02910731, 0.30729987, -1.84214418, 0.00736595, 1.81627148, 0.83959877]
cal_26_mm = [-0.27498185, -0.02917162, 0.30729728, -1.83711943, 0.00740111, 1.81114207, 0.83956440]
cal_28_mm = [-0.27500193, -0.02946890, 0.30729693, -1.83219466, 0.00744732, 1.80599850, 0.83953597]
cal_30_mm = [-0.27501834, -0.02951123, 0.30730128, -1.82733378, 0.00747141, 1.80099033, 0.83951795]
cal_40_mm = [-0.27535423, -0.02973620, 0.30729625, -1.80185362, 0.00748783, 1.77566828, 0.83950376]
cal_50_mm = [-0.27537779, -0.02964236, 0.30729646, -1.77550289, 0.00747043, 1.75009969, 0.83951632]
cal_60_mm = [-0.27538812, -0.02767563, 0.30729625, -1.74815756, 0.00737045, 1.72428787, 0.83968787]
cal_70_mm = [-0.27534187, -0.02516956, 0.30729125, -1.71972291, 0.00653091, 1.69838691, 0.84020591]
cal_80_mm = [-0.27489364, -0.02177300, 0.30729093, -1.69023271, 0.00559191, 1.67217843, 0.84087814]
cal_90_mm = [-0.27442663, -0.01700015, 0.30725260, -1.66094411, 0.00428391, 1.64610852, 0.84159472]
cal_100_mm = [-0.27342677, -0.01169027, 0.30724883, -1.62902436, 0.00271029, 1.61938627, 0.84278577]
cal_110_mm = [-0.27221279, -0.00529769, 0.30724584, -1.59589389, 0.00080301, 1.59226100, 0.84416299]
cal_120_mm = [-0.27068688, 0.00206569, 0.30723810, -1.56146977, -0.00141168, 1.56471790, 0.84578688]
cal_130_mm = [-0.26867866, 0.01051012, 0.30723448, -1.52532189, -0.00398714, 1.53676718, 0.84757781]
cal_140_mm = [-0.26635378, 0.02023801, 0.30722737, -1.48763902, -0.00694801, 1.50828884, 0.84963912]

# calibration 15/02/24 60DEGREE FINGERS EI1 0mm=press
cal_0_mm = [-0.26363617, -0.05473505, 0.30020690, -1.90742241, 0.00734889, 1.87581724, 0.82313380]
cal_2_mm = [-0.26366258, -0.05472722, 0.30021328, -1.90369823, 0.00735250, 1.87093787, 0.82305917]
cal_4_mm = [-0.26371654, -0.05471687, 0.30021328, -1.89936716, 0.00741290, 1.86591738, 0.82294415]
cal_6_mm = [-0.26391464, -0.05504486, 0.30021328, -1.89472156, 0.00748206, 1.86090533, 0.82281917]
cal_8_mm = [-0.26396259, -0.05532080, 0.30021161, -1.88997358, 0.00757329, 1.85586071, 0.82272108]
cal_10_mm = [-0.26403608, -0.05563658, 0.30021328, -1.88528073, 0.00770713, 1.85078742, 0.82262177]
cal_12_mm = [-0.26422203, -0.05593439, 0.30021523, -1.88056663, 0.00779554, 1.84573231, 0.82252450]
cal_14_mm = [-0.26427300, -0.05619965, 0.30021331, -1.87574604, 0.00784266, 1.84068288, 0.82243819]
cal_16_mm = [-0.26438289, -0.05632581, 0.30021690, -1.87089863, 0.00788102, 1.83561542, 0.82236489]
cal_18_mm = [-0.26453586, -0.05657995, 0.30021523, -1.86600990, 0.00791936, 1.83057723, 0.82228960]
cal_20_mm = [-0.26458381, -0.05668035, 0.30021328, -1.86109418, 0.00795603, 1.82553612, 0.82223020]
cal_22_mm = [-0.26463690, -0.05691966, 0.30021328, -1.85616508, 0.00798505, 1.82048298, 0.82216497]
cal_24_mm = [-0.26482269, -0.05695218, 0.30022009, -1.85119037, 0.00800458, 1.81538102, 0.82211627]
cal_26_mm = [-0.26486016, -0.05698160, 0.30021650, -1.84621390, 0.00801417, 1.81027087, 0.82207689]
cal_28_mm = [-0.26489700, -0.05699540, 0.30021845, -1.84108635, 0.00801764, 1.80515880, 0.82204296]
cal_30_mm = [-0.26493536, -0.05700533, 0.30021845, -1.83597784, 0.00801835, 1.80008457, 0.82202409]
cal_40_mm = [-0.26529586, -0.05696333, 0.30022013, -1.80990437, 0.00799617, 1.77460503, 0.82199937]
cal_50_mm = [-0.26544880, -0.05556902, 0.30021846, -1.78293351, 0.00772567, 1.74892897, 0.82204890]
cal_60_mm = [-0.26546751, -0.05339131, 0.30021507, -1.75493735, 0.00707560, 1.72307920, 0.82238232]
cal_70_mm = [-0.26539159, -0.05028930, 0.30021145, -1.72586789, 0.00612781, 1.69704613, 0.82291076]
cal_80_mm = [-0.26503468, -0.04624956, 0.30021437, -1.69573253, 0.00491428, 1.67075798, 0.82362430]
cal_90_mm = [-0.26424438, -0.04132709, 0.30020633, -1.66449699, 0.00342211, 1.64419101, 0.82454922]
cal_100_mm = [-0.26334276, -0.03539899, 0.30020896, -1.63179003, 0.00161609, 1.61728759, 0.82568431]

speed_factor = 0.05 # 0.1 is slow movements, 0.01 is extremely slow

if True: franka.move_joints(cal_10_mm, speed_factor)